# Using YAHPO Gym: A quick introduction

Using YAHPO GYM we can benchmark a new Hyperparameter optimization method on a large amount of problems in a very short time-frame.

This tutorial walks us through the core concepts and functionality of ``yahpo_gym` and showscases a practical example.

YAHPO GYM consists of several `scenarios`, e.g. the collection of all benchmark instances in `lcbench` is a `scenario`.
An `instance` is the concrete task of optimizing hyperparameters of the neural network on a given dataset from OpenML.


## Core functionality: Configuration & BenchmarkSet

We first a have a brief look at at the two core classes we will make use of in `YAHPO GYM`: 
- A `Configuration` contains all relevant infos regarding a specific benchmarking scenario e.g. `lcbench`. We can load configurations with the `cfg(<key>)` shortcut.
- A `BenchmarkSet` can be instantiated using a Configuration (or it's key) and contains the logic used to evaluate the surrogate model for a given query hyperparameter configuration (or set thereof).

### Configuration

In [1]:
# We first load the dict of configurations and the concrete benchmarks
%load_ext autoreload
%autoreload 2
from yahpo_gym.configuration import cfg
import yahpo_gym.benchmarks

In [2]:
# Now we can print a list of available configurations:
print(cfg())

Key             Instances  Cat. HP    Cont. HP   Targets   
lcbench         OpenML_task_id  1          8          6         
fcnet           task            4          8          3         
nb301           CIFAR10         34         1          2         
rbv2_svm        task_id         3          6          6         
rbv2_ranger     task_id         4          7          6         
rbv2_rpart      task_id         2          6          6         
rbv2_glmnet     task_id         2          4          6         
rbv2_xgboost    task_id         3          14         6         
rbv2_aknn       task_id         3          6          6         
rbv2_super      task_id         8          33         6         


In [3]:
# And instantiate a Configuration using a key.
conf_lcb = cfg('lcbench')

This allows us to query several important properties of the benchmark problem:

- config_id : The id / key of the configuration
- y_names  : The names of the target variables included in the surrogate model
- hp_names: The names of all hyperparameters
- cat_names : The names of categorical hyperparameters
- cont_names  :  The names of continuous hyperparameters
- fidelity_params  : The name of the fidelity parameter(s)
- instance_names : The column pertaining to the available instances in a dataset
- runtime_name : The name of parameters remeasuring runtime of  the model. 
- data : A `pandas` `DataFrame` containing the data used to train the surrogates. Only available if the data was downloaded.

In [4]:
# We can for example query the target outputs of our surrogate:
conf_lcb.y_names

['time',
 'val_accuracy',
 'val_cross_entropy',
 'val_balanced_accuracy',
 'test_cross_entropy',
 'test_balanced_accuracy']

### BemchmarkSet

A benchmark set allows us to evaluate the surrogate models for a given configuration.
We can instantiate them similarly to a `Configuration` using the **key**.

In [6]:
from yahpo_gym import benchmark_set
# Select a Benchmark
bench = benchmark_set.BenchmarkSet("lcbench")
bench

BenchmarkInstance (lcbench)

This can again be used to query relevant meta-information:
- instances: The available instances (in this case OpenML Task Id's)

In [9]:
# List available instances
bench.instances[0:5]

['3945', '7593', '34539', '126025', '126026']

We can now set an instance, this defines the instance (i.e. concrete dataset) to be evaluated.
We can furthermore use the included `ConfigSpace` in order to sample a concrete configuration and evaluate it: 

In [10]:
# Set an instance
bench.set_instance("3945")
# Sample a point from the configspace (containing parameters for the instance and budget)
value = bench.config_space.sample_configuration(1).get_dictionary()
# Evaluate
print(bench.objective_function(value))

KeyError: 'epoch'

## A working example

In order to demonstrate using YAHPO Gym more in-depth we provide a full example benchmarking `HPBandSter` on an `lcbench` task.
We again start by importing the relevant modules:

In [12]:
from yahpo_gym import benchmark_set
import yahpo_gym.benchmarks.lcbench
import time
import numpy as np

Now we can define a worker class as required by `HPBandSter` that internally calls our `objective_function`.

In [13]:
from hpbandster.core.worker import Worker
import hpbandster.core.nameserver as hpns
from hpbandster.optimizers import BOHB as BOHB

class lcbench(Worker):

    def __init__(self, *args, sleep_interval=0, **kwargs):
        super().__init__(*args, **kwargs)
        self.bench = bench
        self.sleep_interval = sleep_interval

    def compute(self, config, budget, **kwargs):
        """
        Args:
            config: dictionary containing the sampled configurations by the optimizer
            budget: (float) amount of epochs the model can use to train

        Returns:
            dictionary with mandatory fields:
                "loss" (scalar)
                "info" (dict)
        """

        config.update({"epoch": int(np.round(budget))})  # update epoch
        result = bench.objective_function(config)  # evaluate

        time.sleep(self.sleep_interval)

        return({
                    "loss": - result.get("val_accuracy"),  # we want to maximize validation accuracy
                    "info": "empty"
                })
    
    @staticmethod
    def get_configspace():
        # sets OpenML_task_id constant to "3945" and removes the epoch fidelity parameter
        cs = bench.get_opt_space(instance = "3945", drop_fidelity_params = True)
        return(cs)

ModuleNotFoundError: No module named 'hpbandster'

Using this worker class, we can now run the full benchmark:

In [ ]:

bench = benchmark_set.BenchmarkSet("lcbench")
bench.set_instance("3945")

NS = hpns.NameServer(run_id="lcbench", host="127.0.0.1", port=None)
NS.start()

w = lcbench(sleep_interval=0, nameserver="127.0.0.1", run_id ="lcbench")
w.run(background=True)

bohb = BOHB(configspace=w.get_configspace(),
            run_id="lcbench", nameserver="127.0.0.1",
            min_budget=1, max_budget=52)

res = bohb.run(n_iterations=1)

bohb.shutdown(shutdown_workers=True)
NS.shutdown()

and print the results

In [ ]:
id2config = res.get_id2config_mapping()
id2config

In [ ]:
incumbent = res.get_incumbent_id()
incumbent

In [ ]:

print("Best found configuration:", id2config[incumbent]["config"])
print("A total of %i unique configurations where sampled." % len(id2config.keys()))
print("A total of %i runs where executed." % len(res.get_all_runs()))
print("Total budget corresponds to %.1f full function evaluations."%(sum([r.budget for r in res.get_all_runs()])/1))